In [1]:
import pandas as pd
import numpy as np
import warnings
import statsmodels.api as sm
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks
from sklearn.preprocessing import StandardScaler


pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')

### Import the required libraries and modules that you would need.
### Read that data into Python and call the dataframe churnData.

In [2]:
churnData = pd.read_csv('C:/Users/Ouroboros/lab-handling-data-imbalance-classification/files_for_lab/Customer-Churn.csv')
churnData

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No,Yes,No,No,No,No,Month-to-month,29.85,29.85,No
1,Male,0,No,No,34,Yes,Yes,No,Yes,No,No,No,One year,56.95,1889.5,No
2,Male,0,No,No,2,Yes,Yes,Yes,No,No,No,No,Month-to-month,53.85,108.15,Yes
3,Male,0,No,No,45,No,Yes,No,Yes,Yes,No,No,One year,42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,No,No,No,No,No,Month-to-month,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,Male,0,Yes,Yes,24,Yes,Yes,No,Yes,Yes,Yes,Yes,One year,84.80,1990.5,No
7039,Female,0,Yes,Yes,72,Yes,No,Yes,Yes,No,Yes,Yes,One year,103.20,7362.9,No
7040,Female,0,Yes,Yes,11,No,Yes,No,No,No,No,No,Month-to-month,29.60,346.45,No
7041,Male,1,Yes,No,4,Yes,No,No,No,No,No,No,Month-to-month,74.40,306.6,Yes


### Check the datatypes of all the columns in the data. You would see that the column TotalCharges is object type. Convert this column into numeric type using pd.to_numeric function.

In [3]:
churnData.dtypes

gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [4]:
churnData['TotalCharges'] = pd.to_numeric(churnData['TotalCharges'], errors='coerce')

In [5]:
churnData['TotalCharges'].dtypes

dtype('float64')

### Check for null values in the dataframe. Replace the null values.

In [6]:
churnData.isnull().sum()

gender               0
SeniorCitizen        0
Partner              0
Dependents           0
tenure               0
PhoneService         0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
Contract             0
MonthlyCharges       0
TotalCharges        11
Churn                0
dtype: int64

In [7]:
churnData['TotalCharges'] = churnData['TotalCharges'].fillna(0)

In [8]:
churnData.isnull().sum()

gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

### Use the following features: tenure, SeniorCitizen, MonthlyCharges and TotalCharges:
Scale the features either by using normalizer or a standard scaler.
Split the data into a training set and a test set.
Fit a logistic regression model on the training data.
Check the accuracy on the test data.

In [9]:
from sklearn.preprocessing import StandardScaler

X = churnData[['tenure', 'SeniorCitizen', 'MonthlyCharges']]
y = churnData['TotalCharges']

transformer = StandardScaler().fit(X)
x_standardized = transformer.transform(X)

In [10]:
churnData_standardized = pd.DataFrame(x_standardized,columns=churnData.columns.tolist()[:-1])
churnData_standardized.head()
X = churnData_standardized
y = churnData['TotalCharges']

ValueError: Shape of passed values is (7043, 3), indices imply (7043, 15)

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=100)

In [12]:
classification = LogisticRegression(random_state=100, max_iter=10000)
classification.fit(X_train, y_train)
predictions = classification.predict(X_test)
print(metrics.classification_report(y_test, predictions))

ValueError: Unknown label type: 'continuous'

In [13]:
classification.score(X_test, y_test)

AttributeError: 'LogisticRegression' object has no attribute 'coef_'

### Managing imbalance in the dataset

Check for the imbalance.
Use the resampling strategies used in class for upsampling and downsampling to create a balance between the two classes.
Each time fit the model and see how the accuracy of the model is.

#### Upsampling

In [14]:
churnData.columns

Index(['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure',
       'PhoneService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
       'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract',
       'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')

In [15]:
category_no = churnData[churnData['Churn'] == 'No']
category_yes = churnData[churnData['Churn'] == 'Yes']

category_yes = category_yes.sample(len(category_no), replace=True)  
(category_yes.shape)

(5174, 16)

In [16]:
data2 = pd.concat([category_no, category_yes], axis=0)

data2 = data2.sample(frac=1)
print(data2['Churn'].value_counts())

No     5174
Yes    5174
Name: Churn, dtype: int64


In [17]:
data2

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,MonthlyCharges,TotalCharges,Churn
5939,Male,0,No,No,18,No,Yes,No,Yes,No,No,No,Month-to-month,33.60,550.35,No
2720,Male,0,No,No,15,Yes,No,No,No,No,No,No,Month-to-month,51.55,765.50,Yes
935,Female,1,Yes,No,4,Yes,No,No,No,No,Yes,No,Month-to-month,83.25,308.05,No
5809,Female,0,No,No,6,Yes,No,Yes,No,No,No,Yes,Month-to-month,85.95,514.60,No
5736,Male,0,No,No,1,Yes,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Month-to-month,20.05,20.05,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6862,Female,1,No,No,4,Yes,No,No,Yes,No,No,Yes,Month-to-month,88.85,372.45,Yes
1928,Female,0,Yes,Yes,31,Yes,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Month-to-month,19.55,658.95,Yes
6523,Female,0,No,No,15,No,Yes,Yes,Yes,Yes,No,No,One year,45.90,693.45,No
2321,Female,1,No,No,65,Yes,No,Yes,Yes,No,Yes,No,Month-to-month,95.45,6223.30,No


In [18]:
y = data2['Churn']

numericals = ['tenure', 'SeniorCitizen', 'MonthlyCharges', 'TotalCharges']
numericals = data2[numericals]

In [19]:
transformer = StandardScaler().fit(numericals)
X_stan = transformer.transform(numericals)
X_stan.shape

(10348, 4)

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X_stan, y, test_size=0.3, random_state=42)

In [21]:
classification = LogisticRegression(random_state=42, max_iter=10000)
classification.fit(X_train, y_train)
predictions = classification.predict(X_test)
print(metrics.classification_report(y_test, predictions))

              precision    recall  f1-score   support

          No       0.73      0.73      0.73      1529
         Yes       0.74      0.73      0.74      1576

    accuracy                           0.73      3105
   macro avg       0.73      0.73      0.73      3105
weighted avg       0.73      0.73      0.73      3105



In [22]:
classification.score(X_test, y_test)


0.7339774557165861

#### Downsampling

In [24]:
churnData1 = churnData

In [25]:
cat_no = churnData1[churnData1['Churn'] == 'No']
cat_yes = churnData1[churnData1['Churn'] == 'Yes']


cat_no = cat_no.sample(len(cat_yes))
print(cat_no.shape)
print(cat_yes.shape)

data3 = pd.concat([cat_no, cat_yes], axis=0)

data3 = data3.sample(frac=1)
data3['Churn'].value_counts()

(1869, 16)
(1869, 16)


No     1869
Yes    1869
Name: Churn, dtype: int64

In [27]:
data4 = pd.concat([cat_no, cat_yes], axis=0)

data2 = data2.sample(frac=1)
print(data4['Churn'].value_counts())

No     1869
Yes    1869
Name: Churn, dtype: int64


In [28]:
data4.isnull().values.sum()

0

In [30]:
y = data4['Churn']

numericals = ['tenure', 'SeniorCitizen', 'MonthlyCharges', 'TotalCharges']
numericals = data4[numericals]

In [31]:
transformer = StandardScaler().fit(numericals)
X_stan = transformer.transform(numericals)
X_stan.shape

(3738, 4)

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X_stan, y, test_size=0.3, random_state=42)

In [33]:
classification = LogisticRegression(random_state=42, max_iter=10000)
classification.fit(X_train, y_train)
predictions = classification.predict(X_test)
print(metrics.classification_report(y_test, predictions))

              precision    recall  f1-score   support

          No       0.75      0.74      0.74       578
         Yes       0.73      0.73      0.73       544

    accuracy                           0.74      1122
   macro avg       0.74      0.74      0.74      1122
weighted avg       0.74      0.74      0.74      1122



In [34]:
classification.score(X_test, y_test)

0.7361853832442068